In [1]:
%load_ext pycodestyle_magic

In [2]:
%flake8_on

In [3]:
testdata = """0: 4 1 5
1: 2 3 | 3 2
2: 4 4 | 5 5
3: 4 5 | 5 4
4: "a"
5: "b"

ababbb
bababa
abbbab
aaabbb
aaaabbb""".splitlines()

with open('input', 'r') as inp:
    inputdata = [line.strip() for line in inp.readlines()]

In [4]:
class Rule(object):
    def __init__(self, rule):
        self.literal = None
        self.children = None
        if '"' in rule:
            self.literal = rule[1]
        else:
            if '|' in rule:
                rules = rule.split(' | ', 1)
            else:
                rules = [rule]
            self.children = [[int(r) for r in rule.split()] for rule in rules]

    def __repr__(self):
        if self.literal:
            return self.literal
        else:
            return str(self.children)


def parse2(lines):
    blank_idx = lines.index('')
    rules = parse_rules(lines[:blank_idx])
    messages = lines[blank_idx + 1:]
    return rules, messages


def parse_rules(lines):
    rules = dict()
    for line in lines:
        nr, rule = line.split(': ', )
        rules[int(nr)] = Rule(rule)
    return rules

In [5]:
def match_rule_seq(message, rule_seq, rules):
    if len(message) == 0 and len(rule_seq) == 0:
        return True
    elif len(message) == 0 or len(rule_seq) == 0:
        return False

    if rule_seq[0] not in rules.keys():
        return False
    target = rules[rule_seq[0]]

    if target.literal:
        if message[0] == target.literal:
            return match_rule_seq(message[1:], rule_seq[1:], rules)
        else:
            return False
    elif target.children:
        return any([match_rule_seq(message, seq + rule_seq[1:], rules)
                    for seq in target.children])
    else:
        return False

In [6]:
rules, messages = parse2(inputdata)
# print(rules)

print(sum([match_rule_seq(message, [0], rules) for message in messages]))

rules[8] = Rule('42 | 42 8')
rules[11] = Rule('42 31 | 42 11 31')

print(sum([match_rule_seq(message, [0], rules) for message in messages]))

222
339


## I must have done something right!

-- You may read my desperation beneath --

In [7]:
def parse(data):
    rules = dict()
    messages = []
    section = 1
    for line in data:
        if line == "":
            section += 1
        elif section == 1:
            nr, patterns = [a.strip() for a in line.split(':')]
            if '"' in patterns:
                rules[patterns[-2]] = nr
            elif '|' in patterns:
                p1, p2 = [p.strip() for p in patterns.split('|', 1)]
                rules[p1] = nr
                rules[p2] = nr
            else:
                rules[patterns] = nr
        elif section == 2:
            messages.append(line)
    return rules, messages

In [8]:
rules, messages = parse(testdata)
rules

{'4 1 5': '0',
 '2 3': '1',
 '3 2': '1',
 '4 4': '2',
 '5 5': '2',
 '4 5': '3',
 '5 4': '3',
 'a': '4',
 'b': '5'}

In [9]:
def match(previous_matches, remainder, rules):
    print(f'match({previous_matches}; {remainder})')
    if len(remainder) == 0 and previous_matches == '0':
        return previous_matches  # We are done

    if len(remainder) > 1:
        current, remainder_new = (remainder[0], remainder[1:])
    else:
        current, remainder_new = (remainder, '')  # Soon done

    if current in rules.keys():  # If the currently considered item itself matches, consider those
        if len(previous_matches) == 0:
            matches = match(set(rules[current]), remainder_new, rules)            
        else:
            matches = match([p + rules[current] for p in previous_matches], remainder_new, rules)
            print(f'returning: {matches}')
            return matches

    matches = set()

    if len(previous_matches) == 0:  # If we have no history
        matches |= match(set(current), remainder_new, rules)
    else:
        for p in previous_matches:
            for i in range(1, len(p)+1):
                option = p[-i:] + current
                print(f'option: {option}')
                if option in rules.keys():
                    if len(remainder_new) > 0:
                        if i == len(p):
                            matches = match(set(), rules[option] + remainder_new, rules)
                            print(f'returning: {matches}')
                            return matches
                        else:
                            matches = match(set(p[:-i]), rules[option] + remainder_new, rules)
                            print(f'returning: {matches}')
                            return matches
                    else:
                        if i == len(p):
                            matches = match(set(), rules[option], rules)
                            print(f'returning: {matches}')
                            return matches
                        else:
                            matches = match(set(p[:-i]), rules[option], rules)
                            print(f'returning: {matches}')
                            return matches

    # matches |= set([p + current for p in previous_matches])
    print(f'returning: {matches}')
    return matches

11:80: E501 line too long (98 > 79 characters)
13:71: W291 trailing whitespace
15:80: E501 line too long (97 > 79 characters)
31:80: E501 line too long (88 > 79 characters)
35:80: E501 line too long (94 > 79 characters)


```
abab:
    {a -> 4} 'bab': # assuming existing are empty?
    {4} '5ab'
    {'4 5' -> 3} 'ab'
    {3} '4b'

abab:
    {a -> 4} 'bab': # assuming existing are empty?
    {'4 5'} 'ab'
    {4, 5 -> 3} 'ab'
    {3} '4b'


abbbab:
    a -> 4 'bbbab'
    '4 b -> 5' 'bbab'
    ...
    '4 5 5 5 4 5' ''
    '4 5 5 5' '3'
    '4 5' '2 3'
    '' '3 2 3'
    '1' '1'... #bad

'4 1 5': '0'
    'a {2 3, 3 2} b'

    a bbba b

    
```
        
    

In [10]:
print(f"ab: {match(set(), 'ab', rules)}")
print(f"abab: {match(set(), 'abab', rules)}")
# print(f"{messages[0][0:2]}: {match({'4', 'a'}, messages[0][1], rules)}")
#print(f"{messages[0]} {messages[0][0:3]} {messages[0][3:]}: {match({'34', '3a'}, messages[0][3:], rules)}")
#print(f"{messages[0]}: {match({}, messages[0], rules)}")

4:1: E265 block comment should start with '# '
4:80: E501 line too long (108 > 79 characters)
5:1: E265 block comment should start with '# '
match(set(); ab)
match({'4'}; b)
match(['45']; )
option: 5
option: 45
returning: set()
returning: set()
match({'a'}; b)
match(['a5']; )
option: 5
option: a5
returning: set()
returning: set()
returning: set()
ab: set()
match(set(); abab)
match({'4'}; bab)
match(['45']; ab)
match(['454']; b)
match(['4545']; )
option: 5
option: 45
option: 545
option: 4545
returning: set()
returning: set()
returning: set()
returning: set()
match({'a'}; bab)
match(['a5']; ab)
match(['a54']; b)
match(['a545']; )
option: 5
option: 45
option: 545
option: a545
returning: set()
returning: set()
returning: set()
returning: set()
returning: set()
abab: set()
